In [276]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import f1_score
import warnings

from sys import path # modificar la lista de rutas de búsqueda que el intérprete de Python utiliza para buscar módulos y paquetes.
import os # interactua con el sistema operativo
for dirname, _, filenames in os.walk('../resources/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
path.append(os.path.realpath('../'))

from custom import funtions

../resources/dataset_a_2021.csv
../resources/dataset_a_2021v1.xlsx
../resources/dataset_a_2021v2.xlsx
../resources/dataset_a_2021v3.xlsx
../resources/dataset_a_2021v4.xlsx
../resources/dataset_a_2021v4_col0DExitoSol1.xlsx
../resources/dataset_a_2021v5_ExitoSol1.xlsx
../resources/dataset_a_2021v6_ExitoSol1Programa.xlsx
../resources/dataset_unab_ORIGINAL SIN P1.csv
../resources/dataset_unab_P1.csv


In [277]:
df = pd.read_excel(open('../resources/dataset_a_2021v5_ExitoSol1.xlsx','rb'))

df.head()

,exitosos,fallidos,envios,sol1
0,10,21,31,1.0
1,7,9,16,1.0
2,6,10,16,1.0
3,0,0,0,1.0
4,0,0,0,1.0


In [278]:
df['aprobado']=df.apply(lambda x: funtions.set_in_aprobado_nota(x['sol1']),axis = 1 )
print(df.columns)

Index(['exitosos', 'fallidos', 'envios', 'sol1', 'aprobado'], dtype='object')


In [279]:
# Dividir los datos en características (X) y etiquetas/clases (y)
X = df.drop('sol1', axis=1)  # Característica eliminada
y = df['exitosos']  # Etiquetas/clases a evaluar

In [280]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [281]:
# Crear un modelo de clasificación (por ejemplo, Regresión Logística)
model = LogisticRegression(    
    random_state=0,
    max_iter=10000,
    penalty='l2',
    C=1.0,
    solver='lbfgs',
    tol=0.0001,
    class_weight=None
)

In [282]:
# Entrenar el modelo con el conjunto de entrenamiento
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000, random_state=0)

In [283]:
# Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

In [284]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print('Precisión del modelo: {:.2f}%'.format(accuracy * 100))

Precisión del modelo: 84.52%


In [285]:
precision = accuracy_score(y_test, y_pred)
print("Precisión:", precision)

Precisión: 0.8452380952380952


In [286]:
recall = recall_score(y_test, y_pred, average='macro', zero_division=1)
print("Recall:", recall)


Recall: 0.596877725449154


In [287]:
# Calcular matriz de confusión
confusion = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(confusion)

Matriz de confusión:
[[42  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  5  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  5  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 11  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  5  8  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  7  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 14  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  3  4  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  1  4  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0

In [288]:
# Calcular la puntuación F1 en el conjunto de prueba
f1 = f1_score(y_test, y_pred, average='macro')

print("Puntuación F1:", f1)

Puntuación F1: 0.5279696462221085


In [289]:
# Convertir DataFrame a arreglo NumPy
X = X.values

# Crear instancia de StratifiedKFold con 5 pliegues
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Ignorar el aviso relacionado con la clase menos poblada
warnings.filterwarnings("ignore", category=UserWarning)

# Iterar sobre los pliegues generados por la estratificación
for train_index, test_index in stratified_kfold.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Aquí puedes entrenar y evaluar tu modelo utilizando los conjuntos de entrenamiento y prueba
    # Asegúrate de ajustar tu modelo a X_train y y_train, y evaluarlo en X_test y y_test
    # Por ejemplo:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Exactitud:", accuracy)

Exactitud: 0.8154761904761905
Exactitud: 0.75
Exactitud: 0.7738095238095238
Exactitud: 0.8511904761904762
Exactitud: 0.8203592814371258
